In [ ]:
import numpy as np
import warnings
import pickle
import os
import matplotlib.pyplot as plt
import scipy.stats as st
from utils import utils_gn, utils_noah, utils_ivc, utils_dgrd, utils_models
import importlib
from xgboost import XGBRegressor
importlib.reload(utils_gn)
importlib.reload(utils_noah)
importlib.reload(utils_ivc)
importlib.reload(utils_models)
importlib.reload(utils_dgrd)
warnings.filterwarnings("ignore")

In [ ]:
# load train raw data
train_raw = utils_gn.read_data('train_1238.pkl')

In [ ]:
# Load test data
test_raw = utils_gn.read_data('test_1238.pkl')
y_test_raw = utils_gn.read_data('true_test_labels_1238.pkl')

In [ ]:
# Create sub-sampling time step codes. Here, for instance, mapping a number  
# n to a time step t (in minutes) means that that 1/n of the total 
# length of the CC voltage vector corresponds to taking measurment
# at the end of every time t. Min and max time are 0.05 and 
# 4 minutes respectively.  

def creating_time_steps():

    t = 0.05
    time_steps = []

    while t < 4:
        time_steps.append(t)
        t += 0.05

    return dict(zip(np.arange(len(time_steps))+1, time_steps))


In [ ]:
# Check the codings
creating_time_steps()

In [ ]:
# Cross validation with data from voltage sub-sampling
def CCV_sampling_eol_cross_val():
   
    error_metrics = []
    mae_ci, rmse_ci = [], []

    time_map = creating_time_steps()

    for time in time_map.keys():

        tr = utils_gn.FeatureTransformation(n=50, step_size=time)
        X_train, y_train = tr.fit_transform(data=train_raw, targets=['EOL'], with_eol=True)

        # Build model
        params = {'n_estimators': 50, 'reg_alpha': 0.1, 'max_depth': 2}
        model = XGBRegressor(**params)

        # Call k-fold cross-validation on the training set
        val_scores, val_scores_raw = utils_models.kfold_cross_validation(X=X_train, y=y_train, model=model, cv=3)
        error_metrics.append(list(val_scores.values()))

        # Calculate the 95% CI
        mae_ci.append(utils_models.confidence_interval_any(values=val_scores_raw['test_MAE'], n_bootstraps=10000, alpha=0.05))
        rmse_ci.append(utils_models.confidence_interval_any(values=val_scores_raw['test_RMSE'], n_bootstraps=10000, alpha=0.05))

        print(f'step size: {time_map[time]} done')

    with open(os.path.join("data", "ccv_subsample.pkl"), "wb") as fp:
        pickle.dump((list(time_map.values()), np.array(error_metrics), np.array(mae_ci), np.array(rmse_ci)), fp)

    return list(time_map.values()), np.array(error_metrics), np.array(mae_ci), np.array(rmse_ci)

In [ ]:
time, error, mae_ci, rmse_ci = CCV_sampling_eol_cross_val()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))
ax[0].plot(time[::2], error[:, 0][::2], label='EOL: MAE', color='blue')
ax[0].fill_between(time[::2], mae_ci[:, 0][::2], mae_ci[:, 1][::2], color='blue', alpha=0.15, label='MAE: 95% CI')

ax[1].plot(time[::2], error[:, 1][::2], label='EOL: RMSE', color='crimson')
ax[1].fill_between(time[::2], rmse_ci[:, 0][::2], rmse_ci[:, 1][::2], color='crimson', alpha=0.15, label='RMSE: 95% CI')

ax[0].legend()
ax[1].legend()

ax[0].set_xlabel("Sub-sampling time steps (mins)", fontsize=16)
ax[0].set_ylabel("Cross-validation errors (cycles)", fontsize=16)
ax[1].set_xlabel("Sub-sampling time steps (mins)", fontsize=16)